In [13]:
from subprocess import run
import sys
from pathlib import Path

import cantera as ct
import pandas as pd
import numpy as np
import os
import tqdm

In [14]:
num_pulse = 20
pulse_frequency = 1
file_csv = f'df_strage_{num_pulse}Pulses_{pulse_frequency}kHz.csv'
path_csv = os.path.join('input', file_csv)
print(os.path.exists(path_csv), path_csv)

True input/df_strage_20Pulses_1kHz.csv


In [15]:
file_export = 'reactions_rop_' + file_csv.split('.csv')[0] + '.csv'
path_export = os.path.join('output', file_export)
path_export

'output/reactions_rop_df_strage_20Pulses_1kHz.csv'

In [16]:
gas = ct.Solution('gri30.yaml')

# ROPの行列を取得する

In [17]:
species_names = gas.species_names
reaction_names = gas.reaction_equations()

# 時系列データに対する処理

In [18]:
os.path.exists(path_csv)
df_csv = pd.read_csv(path_csv, index_col=0, dtype=np.float64)
df_csv.head()

,t(sec),P(atm),T(K),H2,H,O,O2,OH,H2O,HO2,...,HCNO,HOCN,HNCO,NCO,N2,AR,C3H7,C3H8,CH2CHO,CH3CHO
0.0,0.000000e+00,20.0,300.000000,0.000000e+00,0.000000e+00,0.002996,0.197728,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.74947,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1.0,1.097389e-12,20.0,300.000046,1.721103e-17,5.373575e-17,0.002996,0.197728,3.824618e-13,4.162079e-20,6.458362e-20,...,1.167008e-63,3.337762e-81,6.041211e-54,2.031863e-65,0.74947,0.0,2.059147e-59,1.376978e-50,1.711932e-46,1.307832e-39
2.0,2.194778e-12,20.0,300.000092,6.883902e-17,2.147191e-16,0.002995,0.197728,7.649022e-13,1.664803e-19,4.684288e-19,...,1.234379e-61,5.050716e-79,3.718805e-52,1.645255e-63,0.74947,0.0,6.338382e-58,3.022498e-49,3.211854e-45,1.602130e-38
3.0,3.840861e-12,20.0,300.000162,2.107966e-16,6.565696e-16,0.002995,0.197728,1.338523e-12,5.098327e-19,2.379744e-18,...,7.362406e-60,5.006591e-77,1.344684e-50,1.019066e-61,0.74947,0.0,1.304886e-56,4.254789e-48,4.040497e-44,1.327402e-37
4.0,4.938250e-12,20.0,300.000208,3.484341e-16,1.084228e-15,0.002995,0.197728,1.720910e-12,8.427701e-19,4.985858e-18,...,4.673282e-59,4.468440e-76,6.893590e-50,7.086728e-61,0.74947,0.0,5.329380e-56,1.438713e-47,1.318249e-43,3.535311e-37


In [19]:
columns_series = reaction_names.copy()
columns_series.insert(0, 't(sec)')
print(columns_series)

['t(sec)', '2 O + M <=> O2 + M', 'H + O + M <=> OH + M', 'H2 + O <=> H + OH', 'HO2 + O <=> O2 + OH', 'H2O2 + O <=> HO2 + OH', 'CH + O <=> CO + H', 'CH2 + O <=> H + HCO', 'CH2(S) + O <=> CO + H2', 'CH2(S) + O <=> H + HCO', 'CH3 + O <=> CH2O + H', 'CH4 + O <=> CH3 + OH', 'CO + O (+M) <=> CO2 (+M)', 'HCO + O <=> CO + OH', 'HCO + O <=> CO2 + H', 'CH2O + O <=> HCO + OH', 'CH2OH + O <=> CH2O + OH', 'CH3O + O <=> CH2O + OH', 'CH3OH + O <=> CH2OH + OH', 'CH3OH + O <=> CH3O + OH', 'C2H + O <=> CH + CO', 'C2H2 + O <=> H + HCCO', 'C2H2 + O <=> C2H + OH', 'C2H2 + O <=> CH2 + CO', 'C2H3 + O <=> CH2CO + H', 'C2H4 + O <=> CH3 + HCO', 'C2H5 + O <=> CH2O + CH3', 'C2H6 + O <=> C2H5 + OH', 'HCCO + O <=> 2 CO + H', 'CH2CO + O <=> HCCO + OH', 'CH2CO + O <=> CH2 + CO2', 'CO + O2 <=> CO2 + O', 'CH2O + O2 <=> HCO + HO2', 'H + O2 + M <=> HO2 + M', 'H + O2 + O2 <=> HO2 + O2', 'H + O2 + H2O <=> HO2 + H2O', 'H + O2 + N2 <=> HO2 + N2', 'H + O2 + AR <=> HO2 + AR', 'H + O2 <=> O + OH', '2 H + M <=> H2 + M', '2 H + H

In [20]:
df_save = pd.DataFrame(index=columns_series)

for i, row in tqdm.tqdm(df_csv.iterrows(), total=df_csv.shape[0]):

    time = row['t(sec)']
    T = row['T(K)']
    P = row['P(atm)']
    X = row['H2':].to_numpy()
    
    # set TPX on solution object
    gas.TPX = T, P*ct.one_atm, X
    net_rop = gas.net_rates_of_progress
    series_rop = pd.Series(net_rop, index=reaction_names)

    series_time = pd.Series([time], index=['t(sec)'])
    series_row = pd.concat([series_time, series_rop])

    df_save = pd.concat([df_save, series_row], axis=1)
    
    # break

df_save.head()

100%|██████████| 26101/26101 [06:17<00:00, 69.15it/s] 


,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
t(sec),0.000000,1.097389e-12,2.194778e-12,3.840861e-12,4.938250e-12,6.035638e-12,8.230416e-12,1.042519e-11,1.310893e-11,1.579267e-11,...,2.302558e+00,2.389096e+00,2.475633e+00,2.562170e+00,2.648707e+00,2.735244e+00,2.821781e+00,2.908319e+00,2.994856e+00,3.020000e+00
2 O + M <=> O2 + M,2020.614205,2.020606e+03,2.020597e+03,2.020584e+03,2.020576e+03,2.020567e+03,2.020550e+03,2.020533e+03,2.020511e+03,2.020490e+03,...,-2.675465e-09,-2.621826e-09,-2.550576e-09,-2.498901e-09,-2.441727e-09,-2.387923e-09,-2.319987e-09,-2.269938e-09,-2.208512e-09,-2.198005e-09
H + O + M <=> OH + M,0.000000,1.510303e-10,6.034904e-10,1.845351e-09,3.047318e-09,4.547378e-09,8.437928e-09,1.350930e-08,2.130412e-08,3.083931e-08,...,-1.237217e-10,-1.212769e-10,-1.179236e-10,-1.155854e-10,-1.129211e-10,-1.104711e-10,-1.072663e-10,-1.050062e-10,-1.021204e-10,-1.016555e-10
H2 + O <=> H + OH,0.000000,1.768000e-16,7.071478e-16,2.165404e-15,3.579283e-15,5.346431e-15,9.940210e-15,1.594609e-14,2.520809e-14,3.657933e-14,...,9.306030e-09,9.006310e-09,8.692806e-09,8.453342e-09,8.156069e-09,7.933751e-09,7.657480e-09,7.458362e-09,7.168616e-09,7.135785e-09
HO2 + O <=> O2 + OH,0.000000,2.553901e-12,1.852356e-11,9.410436e-11,1.971598e-10,3.575511e-10,9.010011e-10,1.825676e-09,3.620511e-09,6.316590e-09,...,-1.405266e-07,-1.380132e-07,-1.345345e-07,-1.320447e-07,-1.292406e-07,-1.265802e-07,-1.232240e-07,-1.207619e-07,-1.176922e-07,-1.171375e-07


In [21]:
df_save.to_csv(path_export)
